In [1]:
import keras as K
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
dataDir = 'Data/'
outputDir = 'Output'

os.makedirs(outputDir,exist_ok=True)

In [29]:
import cv2

X_train = []
y_train = []

X_test = []
y_test = []

dirs = os.listdir('Data/Train')
for d in dirs:
    files = os.listdir('Data/Train/'+d)
    for f in files:
        if(not f.endswith('g')):
            continue
        img = cv2.imread('Data/Train/'+d+'/'+f)
        img = cv2.resize(img,(128,128))
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        X_train.append(img)
        y_train.append(int(d))
        
dirs = os.listdir('Data/Val')
for d in dirs:
    files = os.listdir('Data/Val/'+d)
    for f in files:
        if(not f.endswith('g')):
            continue
        img = cv2.imread('Data/Val/'+d+'/'+f)
        img = cv2.resize(img,(128,128))
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        X_test.append(img)
        y_test.append(int(d))

X_train = np.array(X_train)
y_train = np.array(y_train,dtype=np.int32)
X_test = np.array(X_test)
y_test = np.array(y_test,dtype=np.int32)

print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(5774, 128, 128, 3) (5774,)
(632, 128, 128, 3) (632,)


In [30]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print(y_train.shape,y_test.shape)

(5774, 15) (632, 15)


In [31]:
from keras.models import Sequential
from keras.layers import Conv2D, Activation, MaxPooling2D, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, 3, 3, border_mode='same',
                        input_shape=(128, 128, 3)))
model.add(Activation('relu'))
model.add(Conv2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


model.add(Conv2D(192, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Conv2D(192, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Conv2D(256, 3, 3))
model.add(Activation('relu'))

model.add(keras.layers.GlobalAveragePooling2D())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(15))
model.add(Activation('softmax'))

model.summary()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(128, 128,..., padding="same")`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same")`
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
  from ipykernel import kernelapp as app
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), padding="same")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128,

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_39 (Conv2D)           (None, 128, 128, 32)      896       
_________________________________________________________________
activation_48 (Activation)   (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 126, 126, 32)      9248      
_________________________________________________________________
activation_49 (Activation)   (None, 126, 126, 32)      0         
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 63, 63, 32)        0         
_________________________________________________________________
dropout_24 (Dropout)         (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 63, 63, 64)        18496     
__________

In [37]:
from keras.preprocessing.image import ImageDataGenerator
import cv2
from PIL import Image



datagen = ImageDataGenerator(
    rotation_range=359,
    width_shift_range=0.25,
    height_shift_range=0.25,
    zoom_range=(0.75,1.5),
    fill_mode='nearest',
    cval=0.,
    horizontal_flip=True,
    vertical_flip=True)

val_datagen = ImageDataGenerator(
    fill_mode='nearest',
    cval=0.)

train_generator = datagen.flow(X_train,y_train,
        batch_size=64)

validation_generator = datagen.flow(
        X_test,y_test,
        batch_size=64)

In [38]:
import keras
checkpoint = keras.callbacks.ModelCheckpoint(os.path.join(outputDir,'custom_stage_3_best.h5'), monitor='val_loss',
                                 save_weights_only=False, save_best_only=True)


In [40]:
model_info = model.fit_generator(train_generator,epochs = 48, 
                                 steps_per_epoch= 1+(5774//64),
                                 initial_epoch = 0,
                                 verbose=1,
                                 validation_steps = 1+(632//64),
                                 validation_data =validation_generator,
                                callbacks=[checkpoint])

Epoch 1/48
91/91 [==============================] - 18s 200ms/step - loss: 2.5735 - acc: 0.1928 - val_loss: 2.0191 - val_acc: 0.3528
Epoch 2/48
91/91 [==============================] - 18s 195ms/step - loss: 1.7842 - acc: 0.3827 - val_loss: 1.6521 - val_acc: 0.4177
Epoch 3/48
91/91 [==============================] - 20s 219ms/step - loss: 1.5717 - acc: 0.4369 - val_loss: 1.4796 - val_acc: 0.4778
Epoch 4/48
91/91 [==============================] - 20s 218ms/step - loss: 1.4740 - acc: 0.4614 - val_loss: 1.4189 - val_acc: 0.4810
Epoch 5/48
91/91 [==============================] - 19s 207ms/step - loss: 1.4202 - acc: 0.4788 - val_loss: 1.3293 - val_acc: 0.4984
Epoch 6/48
91/91 [==============================] - 24s 261ms/step - loss: 1.4167 - acc: 0.4733 - val_loss: 1.3209 - val_acc: 0.5190
Epoch 7/48
91/91 [==============================] - 19s 208ms/step - loss: 1.3357 - acc: 0.5083 - val_loss: 1.2680 - val_acc: 0.5206
Epoch 8/48
91/91 [==============================] - 31s 339ms/step - 

KeyboardInterrupt: 

In [41]:
train_generator = datagen.flow(X_train,y_train,
        batch_size=96)

validation_generator = datagen.flow(
        X_test,y_test,
        batch_size=96)

checkpoint = K.callbacks.ModelCheckpoint(os.path.join(outputDir,'custom_stage_2_best.h5'), monitor='val_loss',
                                 save_weights_only=False, save_best_only=True)
model.load_weights(os.path.join(outputDir,'custom_stage_1_best.h5'))
model_info = model.fit_generator(train_generator,epochs = 48, 
                                 steps_per_epoch= 1+(5774//96),
                                 initial_epoch = 0,
                                 verbose=1,
                                 validation_steps = 1+(632//96),
                                 validation_data =validation_generator,
                                callbacks=[checkpoint])

Epoch 1/48
61/61 [==============================] - 20s 320ms/step - loss: 1.0831 - acc: 0.6013 - val_loss: 1.1731 - val_acc: 0.5886
Epoch 2/48
61/61 [==============================] - 20s 325ms/step - loss: 1.0958 - acc: 0.5970 - val_loss: 1.0574 - val_acc: 0.6155
Epoch 3/48
61/61 [==============================] - 35s 579ms/step - loss: 1.0735 - acc: 0.6124 - val_loss: 1.0696 - val_acc: 0.6060
Epoch 4/48
61/61 [==============================] - 19s 313ms/step - loss: 1.0866 - acc: 0.6085 - val_loss: 1.0660 - val_acc: 0.6171
Epoch 5/48
61/61 [==============================] - 34s 559ms/step - loss: 1.0524 - acc: 0.6151 - val_loss: 1.1023 - val_acc: 0.6044
Epoch 6/48
61/61 [==============================] - 19s 315ms/step - loss: 1.0742 - acc: 0.6085 - val_loss: 1.1615 - val_acc: 0.6044
Epoch 7/48
61/61 [==============================] - 22s 365ms/step - loss: 1.0664 - acc: 0.6132 - val_loss: 1.1333 - val_acc: 0.6187
Epoch 8/48
61/61 [==============================] - 35s 574ms/step - 

KeyboardInterrupt: 

In [43]:
model.save('model.h5')